In [1]:
#1. Generate dataset
#2. Train the classifier and save it
#3. Detect the face and named it if it is already stored in the dataset

# Generate dataset

In [1]:
import cv2
import os

import numpy as np 
from PIL import Image

In [3]:
# def generate_dataset():
#     face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
#     def face_cropped(img):
#         gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#         faces = face_classifier.detectMultiScale(gray, 1.3, 5)
#         #scaling factor = 1.3
#         #Minimum neighbor = 5

#         if faces == ():
#             return None
#         for (x, y, w, h) in faces:
#             cropped_face = img[y: y + h, x: x + w]
#         return cropped_face
        
#     cap = cv2.VideoCapture(0)
#     id = 1
#     img_id = 0
    
#     while True:
#         ret, frame = cap.read()
#         if face_cropped(frame) is not None:
#             img_id += 1
#             face = cv2.resize(face_cropped(frame), (200, 200))
#             face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
#             file_name_path = "data/user." + str(id) + "." + str(img_id) + ".jpg" 
#             cv2.imwrite(file_name_path, face)
#             cv2.putText(face, str(img_id), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
#             # (50, 50) is the origin point from where textis to be writen
#             # font scale = 1
#             # thickness = 2
            
#             cv2.imshow("Cropped face", face)
#             if cv2.waitKey(1) == 13 or int(img_id) == 200:
#                 break
#         cap.release()
#         cv2.destroyAllWindows()
#         print("Collecting samples is completed.......")
        
# generate_dataset()

# if not cap.isOpened():
#     print("Error: Could not open video stream.")
#     return

In [4]:
def generate_dataset():
    # Load the cascade classifier for face detection
    face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    
    # Function to crop the face from the image
    def face_cropped(img):
        if img is None:
            return None
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)
        
        if len(faces) == 0:
            return None
        for (x, y, w, h) in faces:
            cropped_face = img[y:y + h, x:x + w]
        return cropped_face

    # Open video capture
    cap = cv2.VideoCapture(0)
    
    # Check if the capture opened successfully
    if not cap.isOpened():
        print("Error: Could not open video stream.")
        return
    
    # Ensure output directory exists
    os.makedirs("data", exist_ok=True)

    id = 3
    img_id = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame")
            break
        
        face = face_cropped(frame)
        if face is not None:
            img_id += 1
            face = cv2.resize(face, (200, 200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            file_name_path = os.path.join("data", f"user.{id}.{img_id}.jpg")
            cv2.imwrite(file_name_path, face)
            cv2.putText(face, str(img_id), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
            
            # Display only the cropped face
            cv2.imshow("Cropped face", face)
            
            # Exit on pressing Enter key or after 200 images
            if cv2.waitKey(1) == 13 or img_id == 200:
                break
    
    # Release resources
    cap.release()
    cv2.destroyAllWindows()
    print("Collecting samples is completed.......")

# generate_dataset()


# Train the classifier and save it

In [6]:
def train_classifier(data_dir):
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
    faces = []
    ids = []

    for image in path:
        img = Image.open(image).convert('L')
        imageNp = np.array(img, 'uint8')
        id = int(os.path.split(image)[1].split(".")[1])

        faces.append(imageNp)
        ids.append(id)

    ids = np.array(ids)

    #Train classifier and save
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces, ids)
    clf.write("classifier.xml")
    
train_classifier("data")

# Detect the face and named it if it is already stored in the dataset 

In [7]:
def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, text,clf):
    gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    features = classifier.detectMultiScale(gray_image, scaleFactor, minNeighbors)
    
    coords = []
    for (x, y, w, h) in features:
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        id, pred = clf.predict(gray_image[y: y + h, x + w])
        confidence = int(100 * (1 - pred / 300))
        
        if confidence > 77:
            if id == 1:
                cv2.putText(img, "Sami", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
            if id == 2:
                cv2.putText(img, "Krisi", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
            if id == 3:
                cv2.putText(img, "Galka", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
        else:
            cv2.putText(img, "UNKNOWN", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, (0, 0, 255), cv2.LINE_AA)
            
        coords = [x, y, w, h]
    return coords

def recognize(img, clf, faceCascade):
    coords = draw_boundary(img, faceCascade, 1.1, 10, (255, 255, 255), "Face", clf)
    return img

faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
clf = cv2.face.LBPHFaceRecognizer_create()
clf.read("classifier.xml")

video_capture = cv2.VideoCapture(0)

while True:
    ret, img = video_capture.read()
    img = recognize(img, clf, faceCascade)
    cv2.imshow("face detection", img)
    
    if cv2.waitKey(1) == 13:
        break
        
video_capture.release()
cv2.destroyAllWindows()

error: OpenCV(4.5.5) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
